In [10]:
import pandas as pd
import numpy as np
import importlib
import time
import os
import warnings
warnings.filterwarnings("ignore")  # Oculta todos los warnings

# Mis modulos
import get_data as gd

In [ ]:
# import pyfiglet
# print(pyfiglet.figlet_format("Autoventa CORP"))

banner = r"""
    _         _                       _           ____ ___  ____  ____  
   / \  _   _| |_ _____   _____ _ __ | |_ __ _   / ___/ _ \|  _ \|  _ \ 
  / _ \| | | | __/ _ \ \ / / _ \ '_ \| __/ _` | | |  | | | | |_) | |_) |
 / ___ \ |_| | || (_) \ V /  __/ | | | || (_| | | |__| |_| |  _ <|  __/ 
/_/   \_\__,_|\__\___/ \_/ \___|_| |_|\__\__,_|  \____\___/|_| \_\_|                                                                          

         💥 AUTOMATIZADOR DE FORMATO AUTOVENTA CORPORATIVO 💥
"""

print(banner, end='\n\n')
time.sleep(1)


    _         _                       _           ____ ___  ____  ____  
   / \  _   _| |_ _____   _____ _ __ | |_ __ _   / ___/ _ \|  _ \|  _ \ 
  / _ \| | | | __/ _ \ \ / / _ \ '_ \| __/ _` | | |  | | | | |_) | |_) |
 / ___ \ |_| | || (_) \ V /  __/ | | | || (_| | | |__| |_| |  _ <|  __/ 
/_/   \_\__,_|\__\___/ \_/ \___|_| |_|\__\__,_|  \____\___/|_| \_\_|                                                                          

          💥 AUTOMATIZADOR DE FORMATO AUTOVENTA CORPORATIVO 💥




In [12]:
# Variables de configuracion
PROJECT_ADDRESS = os.getcwd()

# Diccionarios
FILE_PROPERTIES = {
    'name': os.path.join(PROJECT_ADDRESS, 'query.xlsx'),
    'sheet_name': 'AutovtaSUR',
}
tablas_por_locacion = {} # Guardar los DataFrames por Locacion

# Listas
LOCACIONES = ['06 AYA EL PEDREGAL', '40 AYA CHALA', '88 AYA CAMANA']

In [15]:
importlib.reload(gd)

df_corp = gd.main(FILE_PROPERTIES, LOCACIONES)
print(df_corp.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1675 entries, 0 to 18349
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Locación                 1675 non-null   object
 1   Día                      1675 non-null   object
 2   Texto Breve de Producto  1675 non-null   object
 3   Volumen CF               1675 non-null   object
dtypes: object(4)
memory usage: 65.4+ KB
None


In [16]:
# Obtener FECHAS y PRODUCTOS
FECHAS = sorted(df_corp["Día"].unique())
PRODUCTOS = df_corp["Texto Breve de Producto"].unique()

In [ ]:
# Agrupar por Locacion
for locacion in df_corp["Locación"].unique():
    df_loc = df_corp[df_corp["Locación"] == locacion]

    # Pivotear para que las FECHAS sean columnas
    tabla = df_loc.pivot_table(
        index="Texto Breve de Producto",
        columns=df_loc["Día"],
        values="Volumen CF",
        aggfunc="sum",
        fill_value=0
    )

    # Asegurar que estén todas las FECHAS y PRODUCTOS (incluso si no hubo volumen)
    tabla = tabla.reindex(index=PRODUCTOS, columns=FECHAS, fill_value=0)

    # ❌ Elimina las filas donde el producto no tenga volumen en ninguna fecha
    tabla = tabla.loc[~(tabla == 0).all(axis=1)]

    tablas_por_locacion[locacion] = tabla

In [ ]:
# Guardar las tablas por separado con separador ;
with open(os.path.join(PROJECT_ADDRESS, "query.csv"), "w", encoding="utf-8-sig", newline='') as f:
    for loc, tabla in tablas_por_locacion.items():
        f.write(f"{loc}\n")  # Escribe el nombre de la locación
        tabla_reset = tabla.reset_index()
        tabla_reset.to_csv(f, index=False, sep=";")  # <--- separador punto y coma
        f.write("\n")

### Export it as .py

In [21]:
import nbformat
from nbconvert import PythonExporter

# Cargar el notebook
notebook_path = "main.ipynb"
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Eliminar la última celda
nb['cells'] = nb['cells'][:-1]

# Convertir a script .py
exporter = PythonExporter()
codigo_python, _ = exporter.from_notebook_node(nb)

# Guardar el archivo .py
with open("main.py", "w", encoding="utf-8") as f:
    f.write(codigo_python)

print("✅ Archivo .py creado sin la última celda.")

✅ Archivo .py creado sin la última celda.
